In [36]:
import pandas as pd
from pathlib import Path
import unicodedata
import re

# -------------------------------
# Loaders
# -------------------------------

class DataLoader:
    """
    Clase base para cargar datos desde un archivo.
    
    Parametros:
    -----------
    filepath : str o Path
        Ruta al archivo que se desea cargar.
    """
    def __init__(self, filepath):
        self.filepath = Path(filepath)

    def read_file(self, **kwargs):
        """
        Metodo base que debe ser sobrescrito por cada loader especifico.
        """
        raise NotImplementedError("Este metodo debe ser implementado por subclases")


class CSVLoader(DataLoader):
    """
    Carga archivos CSV.
    """
    def read_file(self, **kwargs):
        return pd.read_csv(self.filepath, **kwargs)


class ExcelLoader(DataLoader):
    """
    Carga archivos Excel (.xls y .xlsx).
    """
    def read_file(self, **kwargs):
        return pd.read_excel(self.filepath, **kwargs)


class JSONLoader(DataLoader):
    """
    Carga archivos JSON.
    """
    def read_file(self, **kwargs):
        return pd.read_json(self.filepath, **kwargs)


# Asociacion entre extension y Loader
LOADER_FACTORY = {
    ".csv": CSVLoader,
    ".xlsx": ExcelLoader,
    ".xls": ExcelLoader,
    ".json": JSONLoader,
    ".txt": CSVLoader
}

# -------------------------------
# Clase para el Data PreProcessor
# -------------------------------

class DataPreprocessor:
    """
    Clase para preprocesamiento de DataFrames.
    
    Parametros:
    -----------
    df : pd.DataFrame
        DataFrame a procesar.
    """
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()

    def clean_columns(self):
        """
        Normaliza los nombres de los columnas:
        - Convierte a minusculas
        - Reemplaza espacios por guoines bajos
        - Elimina caracteres especiales
        - Renombra columnas especificas a nombres estandar
        """
        def limpiar_caracteres(col):
            col = str(col)
            col = unicodedata.normalize("NFKD", col)
            col = col.encode("ascii", "ignore").decode("utf-8")
            col = col.lower()
            col = re.sub(r"\s+", "_", col)
            col = col.replace("@", "a").replace("/", "_")
            col = re.sub(r"[^a-z0-9_]", "", col)
            col = re.sub(r"_+", "_", col).strip("_")
            return col

        self.df.columns = [limpiar_caracteres(c) for c in self.df.columns]

        # Renombrar columnas especificas si existen
        rename_map = {
            "score_15": "score",
            "es_fraude": "fraude",
            "notes_comments": "comentarios"
        }
        self.df.rename(columns={k: v for k, v in rename_map.items() if k in self.df.columns}, inplace=True)
        return self.df

    def vacios_a_nulos(self):
        """
        Convierte valores vacios, espacios en blanco o string 'nan' a valores NaN.
        """
        for col in self.df.columns:
            if self.df[col].dtype == "object":
                # Convertimos a string y reemplazamos espacios vacios por NaN
                self.df[col] = self.df[col].astype("string").replace(r"^\s*$", pd.NA, regex=True)
                # Convertimos string "nan" a NaN
                self.df[col] = self.df[col].replace("nan", pd.NA)
        return self.df

    def limpieza_monto(self, col_name="monto"):
        """
        Limpieza de columnas numericas que representan montos:
        - Elimina simbolos de dolar y comas
        - Convierte a tipo numerico
        """
        if col_name in self.df.columns:
            self.df[col_name] = (
                self.df[col_name].astype(str)
                .str.replace("$", "", regex=False)
                .str.replace(",", "", regex=False)
                .str.strip()
            )
            self.df[col_name] = pd.to_numeric(self.df[col_name], errors="coerce")
        return self.df

    def limpieza_num_cliente(self, col_name="nombre_cliente_raw"):
        """
        Extrae numeros de columnas de clientes y los convierte a numerico.
        """
        if col_name in self.df.columns:
            self.df[col_name] = self.df[col_name].astype(str).str.extract(r"(\d+)")
            self.df[col_name] = pd.to_numeric(self.df[col_name], errors="coerce")
        return self.df

    def limpieza_score(self, col_name="score"):
        """
        Convierte columnas de score de texto a numerico (uno=1, dos=2, etc.).
        """
        if col_name in self.df.columns:
            mapping = {"uno": 1, "dos": 2, "tres": 3, "cuatro": 4, "cinco": 5}
            self.df[col_name] = self.df[col_name].astype(str).str.lower().str.strip()
            self.df[col_name] = self.df[col_name].replace(mapping)
            self.df[col_name] = pd.to_numeric(self.df[col_name], errors="coerce")
        return self.df

    def eliminar_acentos(self):
        """
        Elimina acentos de todas las columnas de tipo string.
        """
        for col in self.df.select_dtypes(include=["string"]).columns:
            self.df[col] = self.df[col].apply(
                lambda x: unicodedata.normalize("NFKD", x).encode("ascii", "ignore").decode("utf-8")
                if pd.notna(x) else x
            )
        return self.df

    def limpieza_fecha_registro(self, col_name="fecha_registro"):
        """
        Convierte columnas de fecha a datetime.
        """
        if col_name in self.df.columns:
            self.df[col_name] = pd.to_datetime(self.df[col_name], errors="coerce")
        return self.df

    def ban_columnas_nulas(self):
        """
        Crea columnas bandera indicando la presencia de valores nulos.
        """
        for col in self.df.columns:
            self.df[f"{col}_nan"] = self.df[col].isna().astype(int)
        return self.df

    def calidad_df(self):
        """
        Retorna informacion basica de calidad del DataFrame:
        - info() del DataFrame
        - resumen de valores nulos
        """
        info = self.df.info()
        null_summary = self.df.isna().sum()
        return {"info": info, "null_summary": null_summary}

# -------------------------------
# Funcion principal
# -------------------------------

def carga_procesa(filepath, **kwargs):
    """
    Carga un archivo y aplica procesamiento estandar:
    - Limpieza de columnas
    - Conversion de vacios a NaN
    - Limpieza de montos, scores y fechas
    - Eliminacion de acentos
    - Creacion de columnas bandera de nulos

    Parametros:
    -----------
    filepath : str o Path
        Ruta al archivo a procesar.
    **kwargs : dict
        Argumentos adicionales para el loader correspondiente (sep, encoding, etc.)

    Retorna:
    --------
    df : pd.DataFrame
        DataFrame procesado.
    calidad : dict
        Informacion de calidad del DataFrame.
    """
    filepath = Path(filepath)
    ext = filepath.suffix.lower()
    if ext not in LOADER_FACTORY:
        raise ValueError(f"Formato no soportado: {ext}")

    # Carga del archivo usando el loader adecuado
    loader = LOADER_FACTORY[ext](filepath)
    df = loader.read_file(**kwargs)

    # Procesamiento de datos
    processor = DataPreprocessor(df)
    processor.clean_columns()
    processor.vacios_a_nulos()
    processor.limpieza_monto()
    processor.limpieza_num_cliente()
    processor.limpieza_score()
    processor.limpieza_fecha_registro()
    processor.eliminar_acentos()
    processor.ban_columnas_nulas()

    return processor.df, processor.calidad_df()

# -------------------------------
# Informacion global del DF
# -------------------------------
if __name__ == "__main__":
    ruta_archivo = input("Ingrese la ruta del archivo: ").strip()
    try:
        df, quality = carga_procesa(ruta_archivo, sep=",", encoding="utf-8")
        print("\nPrimeras filas del dataset:")
        print(df.head(15))
        print("\nCalidad de datos:")
        print(quality["null_summary"])
        print("\nDimension del dataset (filas, columnas):")
        print(df.shape)
    except Exception as e:
        print(f"\nError: {e}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   transaction_id               4543 non-null   float64       
 1   fecha_registro               4643 non-null   datetime64[ns]
 2   nombre_cliente_raw           4513 non-null   float64       
 3   monto                        4539 non-null   float64       
 4   categoria_producto_tipo      4639 non-null   object        
 5   score                        4928 non-null   float64       
 6   comentarios                  4651 non-null   object        
 7   fraude                       4824 non-null   float64       
 8   transaction_id_nan           5000 non-null   int64         
 9   fecha_registro_nan           5000 non-null   int64         
 10  nombre_cliente_raw_nan       5000 non-null   int64         
 11  monto_nan                    5000 non-null 